# NIPALS

- *Auteurs :* Quentin Grimonprez, Cristian Preda, Vincent Vandewalle
- *Date :* 3 février 2020

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
from sklearn.decomposition import PCA

plt.style.use("seaborn")
plt.rcParams["figure.figsize"] = 8, 8
rng = np.random.default_rng(42)  # fixer le seed de l'aléatoire

## Simulation des données

In [ ]:
n = 100
p = 4
mu = [1, 2, 4, 3]
cov = [[0.7, 0, 1.3, 0.5],
       [0, 1.2, -0.3, -0.1],
       [1.3, -0.3, 3.1, 1.3],
       [0.5, -0.1, 1.3, 0.6]]

X = rng.multivariate_normal(mu, cov, n)
X[:10, ]

On vérifie que les données sont "bien" simulées :
- Les moyennes:

In [ ]:
np.round(np.mean(X, axis=0), 2)

- La matrice de variance-covariance :

In [ ]:
np.round(np.cov(X.T), 2)

## ACP normée sur les données simulées.


In [ ]:
sd = X.std(axis=0, ddof=0)
X_scaled = X / sd
X_scaled[:10, ]

In [ ]:
pca = PCA(n_components=4)
pca.fit(X_scaled)

- valeurs propres

In [ ]:
pca.explained_variance_

- facteurs principaux

In [ ]:
pca.components_

- composantes principales

In [ ]:
comp = pca.transform(X_scaled)
comp[:10, :]

Scikit-learn ne propose pas de fonctions graphiques pour l'ACP. Nous écrivons donc ces fonctions.

In [ ]:
def plot_explained_variance(pca, cumulative=False):
    _, ax = plt.subplots()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    if cumulative:
        cumul_ratio = np.cumsum(pca.explained_variance_ratio_)
        ax.bar(range(len(cumul_ratio) + 1), np.concatenate(([0], cumul_ratio)))
        ax.set_ylabel("Cumulative explained variance ratio")
        ax.set_title("Cumulative explained variance ratio")
    else:
        ax.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
        ax.set_ylabel("Explained variance ratio")
        ax.set_title("Explained variance ratio")

    ax.set_xlabel("Number of components")

    return ax


def plot_individuals(score, comp=[0, 1]):
    _, ax = plt.subplots()
    ax.axhline(y=0, color="k", lw=1, ls="--")
    ax.axvline(x=0, color="k", lw=1, ls="--")
    ax.scatter(score[:, comp[0]], score[:, comp[1]])
    ax.set_xlabel("Dim " + str(comp[0]))
    ax.set_ylabel("Dim " + str(comp[1]))
    ax.set_title("PCA graph of individuals")

    return ax


def get_var_contribution(pca):
    return pca.components_ * np.sqrt(pca.explained_variance_[:pca.n_components_]).reshape(-1, 1)


def plot_variables(pca, comp=[0, 1], label=None):
    _, ax = plt.subplots()

    if label is None:
        label = ["V" + str(i) for i in range(pca.n_features_)]

    var_contrib = get_var_contribution(pca)

    # plot circle + axis
    an = np.linspace(0, 2 * np.pi, 100)
    ax.plot(np.cos(an), np.sin(an), color="k")
    ax.axhline(y=0, color="k", lw=1, ls="--")
    ax.axvline(x=0, color="k", lw=1, ls="--")

    # plot arrow + text
    for i in range(pca.n_features_):
        ax.arrow(0, 0,
                  var_contrib[comp[0], i], var_contrib[comp[1], i],
                  head_width=0.01, head_length=0.01)

        ax.text(var_contrib[comp[0], i] + 0.025,
                 var_contrib[comp[1], i] + 0.025,
                 label[i])

    ax.set_xlabel("Dim " + str(comp[0]) + " (" + str(round(pca.explained_variance_ratio_[comp[0]] * 100, 2)) + "%)")
    ax.set_ylabel("Dim " + str(comp[1]) + " (" + str(round(pca.explained_variance_ratio_[comp[1]] * 100, 2)) + "%)")
    ax.set_title("PCA graph of variables")

    return ax

In [ ]:
plot_explained_variance(pca, cumulative=False)

In [ ]:
plot_individuals(comp, [0, 1])

In [ ]:
plot_variables(pca)

### NIPALS sans traitement des données manquantes

In [ ]:
def NIPALS(X, h=2, iter=100):
    # renvoie les composantes principales (CP), les facteurs principaux (FP) et les données reconstituées avec h composantes (Xrec)
    n, p = X.shape

    # centrer et réduire matrice X
    m = X.mean(axis=0)
    s = X.std(axis=0, ddof=0)
    X_reduit = (X - m) / s

    # on réserve la place pour:
    CP = np.zeros((n, h))  # les composantes principales
    FP = np.zeros((h, p))  # les facteurs principaux
    X_recons = np.zeros((n, p))  # les données reconstituées

    # déroulement de l'algorithme:
    for i in range(h):
        # voir pages 30-32 du cours
        cp, fp = calcul_cp_fp(X_reduit, iter)  # fonction qui calcule la 1ere comp. princ et 1er fact. principal
        CP[:, i] = cp
        FP[i, :] = fp
        X_reduit -= np.matmul(cp.reshape(-1, 1), fp.reshape(1, -1))

    # Reconstitution des données avec h composantes
    X_recons = np.matmul(CP, FP)
    X_recons *= s
    X_recons += m

    return CP, FP, X_recons


def calcul_cp_fp(X, iter=100):
    cp = X[:, 0].copy()
    fp = np.zeros((X.shape[1], ))
    for i in range(iter):
        fp = np.matmul(X.T, cp)

        # on normalize fp:
        fp /= np.sqrt(np.sum(fp**2))
        cp = np.matmul(X, fp)

    return cp, fp

### Application de NIPALS aux données simulées
Voici ce qu'on obtient avec h=4 composantes. À comparer avec ce qui est donné par scikit-learn dans l'objet 'pca'.

In [ ]:
cp, fp, X_rec = NIPALS(X, h=p)

Les facteurs principaux :
- nipals

In [ ]:
fp

- scikit-learn

In [ ]:
pca.components_

Les composantes principales :
- nipals

In [ ]:
cp[:6, :]

- scikit-learn

In [ ]:
comp[:6, :]

#### La reconstitution complète des données (toutes les composantes principales) 

La reconstitution des données avec toutes les composantes principales.
- Données reconstituées

In [ ]:
X_rec[:6, :]

- Données

In [ ]:
X[:6, :]

#### Approximation des données avec quelques composantes (ici h=2)

Voici la reconstitution des données avec juste deux composantes :

In [ ]:
cp, fp, X_rec = NIPALS(X, h=2)

- Données reconstituées

In [ ]:
X_rec[:6, :]

- Données

In [ ]:
X[:6, :]

## NIPALS avec données manquantes.
L'algorithme précédent est adapté aux cas où il n'y a pas de données manquantes. Les points à modifier sont au niveau du :

  - calcul des moyennes (m) et écart-types (s)
  - calcul des composantes et facteurs dans la fonction calcul_cp_fp

On ré-écrit donc ces fonctions en les renommant : *NIPALS_dm* et *calcul_cp_fp_dm*

In [ ]:
def NIPALS_dm(X, h=2, iter=100):
    # renvoie les composantes principales (CP), les facteurs principaux (FP) et les données reconstituées avec h composantes (Xrec)
    n, p = X.shape

    # centrer et réduire matrice X
    m = np.nanmean(X, axis=0)
    s = np.nanstd(X, axis=0, ddof=0)
    X_reduit = (X - m) / s

    # on réserve la place pour:
    CP = np.zeros((n, h))  # les composantes principales
    FP = np.zeros((h, p))  # les facteurs principaux
    X_recons = np.zeros((n, p))  # les données reconstituées

    # déroulement de l'algorithme:
    for i in range(h):
        # voir pages 30-32 du cours
        cp, fp = calcul_cp_fp_dm(X_reduit, iter)  # fonction qui calcule la 1ere comp. princ et 1er fact. principal
        CP[:, i] = cp
        FP[i, :] = fp
        X_reduit -= np.matmul(cp.reshape(-1, 1), fp.reshape(1, -1))

    # Reconstitution des données avec h composantes
    X_recons = np.matmul(CP, FP)
    X_recons *= s
    X_recons += m

    return CP, FP, X_rec


def calcul_cp_fp_dm(X, iter=100):
    cp = X[:, 0].copy()
    fp = np.zeros((X.shape[1], ))
    for i in range(iter):
        for j in range(X.shape[1]):
            fp[j] = np.nansum(X[:, j] * cp)

        # on normalize fp:
        fp /= np.sqrt(np.sum(fp**2))

        for j in range(X.shape[0]):
            cp[j] = np.nansum(X[j, :] * fp)

    return cp, fp

On vérifie que la version modifiée *NIPALS_dm* donne les mêmes résultats que *NIPALS* lorsqu'il n'y a pas données manquantes. 

In [ ]:
cp_dm, fp_dm, X_rec_dm = NIPALS_dm(X, h=p)
cp, fp, X_rec = NIPALS(X, h=p)

Les facteurs principaux :

In [ ]:
fp_dm

In [ ]:
fp

Les composantes principales :

In [ ]:
cp_dm[:6, :]

In [ ]:
cp[:6, :]

Les données reconstituées :

In [ ]:
X_rec_dm[:6, :]

In [ ]:
X_rec[:6, :]

Parfait!

### Simulation des données manquantes sur la matrice X


In [ ]:
# pourcentage des données manquantes
prop_miss = 0.1
# génération des valeurs manquantes
X_miss = X.copy()
ind = rng.random((n, p))
is_missing = ind < prop_miss
X_miss[is_missing] = np.nan

Nombre de manquants par colonne :

In [ ]:
np.isnan(X_miss).sum(axis=0)

Voici les valeurs qui ont été déclarées manquantes :

In [ ]:
X[is_missing]

### Imputation des valeurs manquantes avec NIPALS :

In [ ]:
cp_imp, fp_imp, X_rec_imp = NIPALS_dm(X_miss, p)

Voici les valeurs estimées par NIPALS pour les données manquantes :

In [ ]:
X_rec_imp[is_missing]

Cela a l'air pas mal!